In [3]:
import pandas as pd
df = pd.read_csv('/Users/user/Desktop/0model/0final_cleaned_dataset.csv')
df.head()  # Shows the first 5 rows


,Header_Length,Protocol Type,Time_To_Live,Rate,fin_flag_number,syn_flag_number,rst_flag_number,psh_flag_number,ack_flag_number,ack_count,...,Min,Max,AVG,Std,Tot size,IAT,Number,Variance,Label,BroadLabel
0,3.044522,1.945910,4.174387,10.939128,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,4.110874,4.110874,4.110874,0.000000,4.110874,0.000018,4.615121,0.000000,DDOS-TCP_FLOOD,DDOS
1,3.050220,1.945910,4.174387,9.935243,0.0,0.0,0.0,0.009950,0.009950,0.693147,...,4.110874,6.812345,4.241039,4.452019,4.241039,0.000048,4.615121,8.880730,DOS-TCP_FLOOD,NON-DDOS
2,3.044522,1.945910,4.238445,3.606232,0.0,0.0,0.0,0.182322,0.405465,1.791759,...,4.204693,5.846439,5.213848,4.767263,5.213848,0.028876,2.397895,9.517519,BENIGN,BENIGN
3,3.496508,1.945910,5.107157,5.708131,0.0,0.0,0.0,0.095310,0.693147,2.397895,...,4.204693,7.993958,6.484941,6.915838,6.484941,0.003348,2.397895,13.829692,BENIGN,BENIGN
4,2.879198,2.890372,3.914021,4.525587,0.0,0.0,0.0,0.095310,0.336472,1.609438,...,4.110874,5.389072,4.746670,4.059949,4.746670,0.014429,2.397895,8.085405,BENIGN,BENIGN


In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, LSTM, BatchNormalization, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

In [ ]:
# Step 2: Create Binary Labels (Benign vs Attack)
df['Binary_Label'] = df['Label'].apply(lambda x: 'BENIGN' if x == 'BENIGN' else 'ATTACK')

# Confirm the label distribution
print(df['Binary_Label'].value_counts())


In [11]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['Binary_Label_encoded'] = label_encoder.fit_transform(df['Binary_Label'])

# BENIGN → 0, ATTACK → 1
print(dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))


{'ATTACK': 0, 'BENIGN': 1}


In [13]:
# Drop unnecessary columns
columns_to_drop = ['Label', 'BroadLabel', 'Binary_Label']
feature_columns = df.drop(columns=columns_to_drop + ['Binary_Label_encoded']).select_dtypes(include=['number']).columns.tolist()

# Final feature set and label
X = df[feature_columns]
y = df['Binary_Label_encoded']

print("✅ Feature shape:", X.shape)


✅ Feature shape: (2323046, 28)


In [15]:
from sklearn.model_selection import train_test_split

# Split: 60% train, 20% val, 20% test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print(f"✅ Train: {X_train.shape}, Validation: {X_val.shape}, Test: {X_test.shape}")


✅ Train: (1393827, 28), Validation: (464609, 28), Test: (464610, 28)


In [17]:
from tensorflow import keras

model = keras.models.load_model('/Users/user/Downloads/best_model1.h5')


In [19]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("✅ Data scaled.")


ValueError: Input X contains infinity or a value too large for dtype('float64').

In [21]:
# Replace inf with NaN, then fill NaN with mean
X_train = X_train.replace([np.inf, -np.inf], np.nan)
X_val = X_val.replace([np.inf, -np.inf], np.nan)
X_test = X_test.replace([np.inf, -np.inf], np.nan)

# Fill missing values with column mean
X_train = X_train.fillna(X_train.mean())
X_val = X_val.fillna(X_train.mean())  # Use training mean for validation/test
X_test = X_test.fillna(X_train.mean())


In [23]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("✅ Data scaled.")


✅ Data scaled.


In [25]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  # Use categorical_crossentropy if y_train is one-hot encoded
    metrics=['accuracy']
)


In [31]:

x_train=X_train_scaled
x_val=X_val_scaled
x_test=X_test_scaled


history = model.fit(
    x_train, y_train,
    epochs=3,
    validation_split=0.2  # use 20% of training data as validation
)


Epoch 1/3


2025-05-07 08:26:39.711546: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at sparse_xent_op.cc:103 : INVALID_ARGUMENT: Received a label value of 1 which is outside the valid range of [0, 1).  Label values: 1 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 0 0 1 0 1 1 1 1 0 1 0 0 0 1 1 1
2025-05-07 08:26:39.712061: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Received a label value of 1 which is outside the valid range of [0, 1).  Label values: 1 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 0 0 1 0 1 1 1 1 0 1 0 0 0 1 1 1
	 [[{{function_node __inference_one_step_on_data_5490}}{{node compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits}}]]


InvalidArgumentError: Graph execution error:

Detected at node compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 701, in start

  File "/opt/anaconda3/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/opt/anaconda3/lib/python3.12/asyncio/base_events.py", line 641, in run_forever

  File "/opt/anaconda3/lib/python3.12/asyncio/base_events.py", line 1987, in _run_once

  File "/opt/anaconda3/lib/python3.12/asyncio/events.py", line 88, in _run

  File "/o

In [35]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    x_train, y_train,
    validation_split=0.2
)

   83/34846 ━━━━━━━━━━━━━━━━━━━━ 7:31:25 779ms/step - accuracy: 0.9609 - loss: 0.1490

KeyboardInterrupt: 